In [ ]:
#from __future__ import print_function
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn import metrics
import os
import glob  # to count jpg files

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras import regularizers, optimizers
import keras
from sklearn.metrics import confusion_matrix
import itertools
from tensorflow.keras.applications import VGG19, ResNet50, xception, inception_v3

import time
import winsound  # to play a sound when the program finishes


def createSimpleModelTune(dr1, dr3, noOfFilters, kernelSize):
    model = Sequential()
    # The first two layers with 32 filters of window size 3x3
    model.add(Conv2D(noOfFilters, (kernelSize, kernelSize), padding='same', activation='relu', input_shape=(128, 128, 3)))#model.add(Conv2D(8, (7, 7), padding='same', activation='relu', input_shape=(128, 128, 3)))
    # model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dr1))  # 0.25


    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(dr3))  # 0.9
    model.add(Dense(10, activation='softmax',
                    kernel_regularizer=regularizers.l1(0.01),
                    activity_regularizer=regularizers.l1(0.01)))

    return model

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.savefig('D:/cfValCCAT10.pdf', bbox_inches = 'tight')
    plt.show()
    return

def CCAT10Case(justLoadPreviousModel = True):


    image_size = 128
    noOfRuns = 1
    epochs = 1000
    batch_size = 32
    '''
    # dropouts initial CCAT
    dr1 = 0.3
    dr2 = 0.6
    dr3 = 0.5
    '''
    # dropouts from IMDB
    dr1 = 0.3
    dr2 = 0.6
    dr3 = 0.8

    mainPath = 'D:\\Datasets\\CCAT\\CCAT-GAN\\'
    trainPath = mainPath + 'train'
    validPath = mainPath + 'valid'
    testPath = mainPath + 'test'

    jpgCounterTrain = len(glob.glob(trainPath + "/**/*.jpg")) + len(glob.glob(trainPath + "/**/*.png")) + len(glob.glob(trainPath + "/**/*.tiff"))
    jpgCounterValid = len(glob.glob(validPath + "/**/*.jpg")) + len(glob.glob(validPath + "/**/*.png")) + len(glob.glob(validPath + "/**/*.tiff"))
    jpgCounterTest = len(glob.glob(testPath + "/**/*.jpg")) + len(glob.glob(testPath + "/**/*.png")) + len(glob.glob(testPath + "/**/*.tiff"))



    lossValue = 'categorical_crossentropy'
    classModeValue = 'categorical'

    start_time = time.time()
    i = 0
    dr1 = 0.5
    dr3 = 0.3
    noOfFilters = 16
    kernelSize = 7
    '''
    for dr1 in [0.1, 0.5]:
        for dr3 in [0.1, 0.3]:
            for noOfFilters in [4, 8, 16, 32]:
                for kernelSize in [3, 5, 7, 9]:
    '''
    #model1 = createSimpleModel(dr1, dr3)
    model1 = createSimpleModelTune(dr1, dr3, noOfFilters, kernelSize)

    model1.compile(loss=lossValue, optimizer="rmsprop",
                   metrics=['acc'])  # 1e-6 #optimizers.RMSprop(lr=1e-6, momentum=0.9)
    # this is the augmentation configuration we will use for training
    train_datagen = ImageDataGenerator(rescale=1. / 255)
    # this is the augmentation configuration we will use for testing:
    # only rescaling
    valid_datagen = ImageDataGenerator(rescale=1. / 255)
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    seed = 5

    # this is a generator that will read pictures found in
    # subfolDers of 'data/train', and indefinitely generate
    # batches of augmented image data
    train_generator = train_datagen.flow_from_directory(
        trainPath,  # this is the target directory
        target_size=(image_size, image_size),
        batch_size=batch_size,
        #color_mode='grayscale',
        class_mode=classModeValue, seed=seed,
        shuffle=True)  # since we use binary_crossentropy loss, we need binary labels

    # this is a similar generator, for validation data
    validation_generator = valid_datagen.flow_from_directory(
        validPath,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        #color_mode='grayscale',
        class_mode=classModeValue, shuffle=False)

    test_generator = test_datagen.flow_from_directory(
        testPath,
        target_size=(image_size, image_size),
        batch_size=jpgCounterTest,
        #color_mode='grayscale',
        class_mode=classModeValue, shuffle=False)

    outputFolder = 'D:\\Datasets\\Saved2020Results\\CCAT-2020-Revision\\'
    # create the folder
    if not os.path.exists(outputFolder):
        os.makedirs(outputFolder)

    filepath = outputFolder + "AugmentedWeightsBestRunCCAT10.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [EarlyStopping(monitor='val_acc', patience=50, verbose=0), checkpoint]

    if justLoadPreviousModel == False:
        history2 = model1.fit_generator(
            train_generator,
            steps_per_epoch=jpgCounterTrain // batch_size,
            epochs=epochs,
            validation_data=validation_generator,
            callbacks=callbacks_list,
            validation_steps=jpgCounterValid // batch_size)

        plt.figure(figsize=[8, 6])
        plt.plot(history2.history['loss'], 'r', linewidth=3.0)
        plt.plot(history2.history['val_loss'], 'b', linewidth=3.0)
        plt.legend(['Training loss', 'Validation Loss'], fontsize=18)
        plt.xlabel('Epochs ', fontsize=16)
        plt.ylabel('Loss', fontsize=16)
        plt.title('Loss Curves', fontsize=16)
        plt.show()

        # Accuracy Curves
        plt.figure(figsize=[8, 6])
        plt.plot(history2.history['acc'], 'r', linewidth=3.0)
        plt.plot(history2.history['val_acc'], 'b', linewidth=3.0)
        plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=18)
        plt.xlabel('Epochs ', fontsize=16)
        plt.ylabel('Accuracy', fontsize=16)
        plt.title('Accuracy Curves', fontsize=16)
        plt.show()
    elapsedTrainingTime = time.time() - start_time

    model1.load_weights(filepath)

    xTest, yTest = test_generator.next()
    print('class_indices = ', test_generator.class_indices)
    evaluation = model1.evaluate(xTest, yTest)
    print("Loss and evaluation on the test set:", evaluation)

    if justLoadPreviousModel == False:
    # save results to file
    # import os.path
        outputFileName = outputFolder + '\AugmentedResultsCNNKernelTuneFiltersKS.txt'
        if os.path.isfile(outputFileName):  # if the file exists, add to it, otherwise create it
            f = open(outputFileName, 'a')

            f.write(str(i) + '\t' + str(dr1) + '\t' + str(dr2) + '\t' + str(dr3)
                    + '\t' + str(noOfFilters) + '\t' + str(kernelSize)
                    + '\t' + str(epochs)
                    + '\t' + str(batch_size) + '\t' + str(image_size)
                    + '\t' + str(noOfRuns) + '\t' + str(jpgCounterValid) + '\t' + str(jpgCounterTest)
                    + '\t' + str(evaluation[0]) + '\t' + str(evaluation[1])
                    + '\t' + str(min(history2.history['loss'])) + '\t' + str(max(history2.history['acc']))
                    + '\t' + str(min(history2.history['val_loss'])) + '\t' + str(max(history2.history['val_acc']))
                    + '\t' + str(elapsedTrainingTime) + '\n')
            f.close()  # I did not test if these close() are necessary. It works without them
        else:
            f = open(outputFileName, 'w')
            f.write(
                'currentRun\tdr1\tdr2\tdr3\t\tnoOfFilters\tkernelSize\tepochs\tbatch_size\timage_size\tnoOfRuns\tvalidationImages\ttestImages\ttestLoss\ttestAccuracy\tminTrainLoss\tmaxTrainAcc\tminValidLoss\tmaxValidAcc\telapsedTrainingTime\n')
            f.close()
            f = open(outputFileName, 'a')
            f.write(str(i) + '\t' + str(dr1) + '\t' + str(dr2) + '\t' + str(dr3)
                    + '\t' + str(noOfFilters) + '\t' + str(kernelSize)
                    + '\t' + str(epochs)
                    + '\t' + str(batch_size) + '\t' + str(image_size)
                    + '\t' + str(noOfRuns) + '\t' + str(jpgCounterValid) + '\t' + str(jpgCounterTest)
                    + '\t' + str(evaluation[0]) + '\t' + str(evaluation[1])
                    + '\t' + str(min(history2.history['loss'])) + '\t' + str(max(history2.history['acc']))
                    + '\t' + str(min(history2.history['val_loss'])) + '\t' + str(max(history2.history['val_acc']))
                    + '\t' + str(elapsedTrainingTime) + '\n')
            f.close()
        # save acc and loos for train and validation during evolution
        outputFileRuntime = outputFolder + '\AugmentedMoreResultsRuntimeCNN' + str(dr1) + str(dr2) + str(dr3) + '.txt'
        print('The following file is saved: ', outputFileRuntime)
        np.savetxt(outputFileRuntime, np.c_[
            history2.history['loss'], history2.history['val_loss'], history2.history['acc'], history2.history[
                'val_acc']])

    # %%

    entireValidationGenerator = valid_datagen.flow_from_directory(
        validPath,
        target_size=(image_size, image_size),
        batch_size=jpgCounterValid,
        #color_mode='grayscale',
        class_mode=classModeValue, shuffle=False)

    xVal, yVal = entireValidationGenerator.next()

    test_generator = test_datagen.flow_from_directory(
        testPath,
        target_size=(image_size, image_size),
        batch_size=jpgCounterTest,
        #color_mode='grayscale',
        class_mode=classModeValue, shuffle=False)
    xTest, yTest = test_generator.next()

    evaluationVal = model1.evaluate(xVal, yVal)
    print("Loss and evaluation on the validation set:", evaluationVal)
    predictionsVal = model1.predict(xVal)

    predictionsTest = model1.predict(xTest)
    evaluationTest = model1.evaluate(xTest, yTest)
    print("Loss and evaluation on the test set:", evaluationTest)

    cmVal = confusion_matrix(yVal.argmax(1), predictionsVal.argmax(axis=-1))
    print('Val CF:\n', cmVal)

    cmTest = confusion_matrix(yTest.argmax(1), predictionsTest.argmax(axis=-1))
    print('Test CF:\n', cmTest)
    print('Evaluation of validation')
    print(metrics.classification_report(yVal.argmax(1), predictionsVal.argmax(axis=-1), digits=3))
    print('Evaluation of test')
    print(metrics.classification_report(yTest.argmax(1), predictionsTest.argmax(axis=-1), digits=3))
    cmVal = confusion_matrix(yVal.argmax(1), predictionsVal.argmax(axis=-1))
    cm_plot_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
    plot_confusion_matrix(cmVal, cm_plot_labels, title='Confusion Matrix Validation Set')

    cmTest = confusion_matrix(yTest.argmax(1), predictionsTest.argmax(axis=-1))
    cm_plot_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
    plot_confusion_matrix(cmTest, cm_plot_labels, title='Confusion Matrix Test Set')
    print('Actual outputs:', yTest.argmax(1))
    print('Predicted outputs:', predictionsTest.argmax(axis=-1))
    print('dr1 = {}, dr3 = {}'.format(dr1, dr3))
    keras.backend.clear_session()
    return evaluationTest

evaluationTest = CCAT10Case(False)
